In [1]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import keys
import json
import datetime
import functions as f
from tqdm import tqdm
import time

In [2]:
state = pd.read_csv('states.csv')

In [3]:
states = list(state.Abbreviation)

In [15]:
size = '200' #maxsize
engine = db.create_engine('sqlite:///tickets.db')
table = 'events'
frame_list = []
fails = []
success= []


In [ ]:
frame_list = []
for state in states:
    for page in range(5):
        df = f.scrape_ticketmaster(df)
        df = f.drop_columns(df)
        df = f.unpack_classifications(df)
        df = f.unpack_presales(df)
        df = f.unpack_price(df)
        df = f.unpack_limit(df)
        df = f.unpack_venue(df)
        df = f.unpack_market(df)
        df = f.final_cleanup(df)

In [23]:
frames = frame_list.copy()

In [101]:
df.to_csv('tm_events')

In [105]:
df = pd.read_csv('tm_events.csv')

In [106]:
venues = list(df.venue_name)

In [109]:
venues = list(set(venues))

In [110]:
len(venues)

2779